# Python manages, C works

When exchanging information between Python and C, we don't want to duplication precious resources.  We want a resource manager to do bookkeeping and interpretation on demand.  When the resource is a simple array, a pointer plus an integer will do.  But when it's more complex than the trivial case, we'll have more work to do.

This module demonstrates how to code up a multidimensional array that allows negative indices in C.  There will be three steps to achieve the goal.

# Step 1: Make C struct available

We want to manage memory in Python, so let's start with a pure Python class.

In [2]:
from __future__ import absolute_import, division, print_function

import numpy as np

class GhostArray:
    def __init__(self, *args, **kw):
        # Pop all custom keyword arguments.
        creator_name = kw.pop("creation", "empty")
        # Create the ndarray and thus control the life cycle.
        create = getattr(np, creator_name)
        self.nda = create(*args, **kw)
        if not self.nda.flags.c_contiguous:
            raise ValueError("not C contiguous")
        ndim = len(self.nda.shape)
        if ndim == 0:
            raise ValueError("zero dimension is not allowed")
        assert ndim > 0

    def __getattr__(self, name):
        return getattr(self.nda, name)

In [3]:
# Use ellipsis to set the contents.
grr = GhostArray(5, dtype="int32")
grr.nda[...] = np.arange(0, grr.size*2, 2, dtype="int32")
print(grr.nda, grr)

[0 2 4 6 8] <__main__.GhostArray object at 0x10441ecc0>


## Prepare a C struct in Cython

In [4]:
%load_ext Cython

In [7]:
%%cython

# Import from Cython-supplied clib header.
from libc.stdlib cimport malloc, free

import numpy as np
cimport numpy as np


# Define a C struct.
ctypedef struct gstbuf_t:
    char *elem
    np.npy_intp nelem
    int elemsize


# Define a C extension type.
cdef class GhostArray:
    # Members of the extension type should be declared.
    cdef gstbuf_t *_data
    # Overwrite this memory holder may result into segfault with _data.elem.
    cdef readonly object nda

    # C-level "constructor"
    def __cinit__(self, *args, **kw):
        self._data = <gstbuf_t*>malloc(sizeof(gstbuf_t))
        self._data.elem = <char*>(NULL)
        self._data.elemsize = 0

    # C-level "destructor"
    def __dealloc__(self):
        if NULL != self._data:
            # Don't touch self._data.elem, which is managed by self.nda.
            free(self._data)

    def __init__(self, *args, **kw):
        # Pop all custom keyword arguments.
        creator_name = kw.pop("creation", "empty")
        # Create the ndarray and thus control the life cycle.
        create = getattr(np, creator_name)
        self.nda = create(*args, **kw)
        if not self.nda.flags.c_contiguous:
            raise ValueError("not C contiguous")
        ndim = len(self.nda.shape)
        if ndim == 0:
            raise ValueError("zero dimension is not allowed")
        assert ndim > 0
        # Initialize internal data.
        ## elem.
        cdef np.ndarray cnda = self.nda
        self._data.elem = <char*>cnda.data
        ## nelem.
        self._data.nelem = self.nda.size
        ## elemsize (just a duplication of PyArray_Descr.elsize).
        self._data.elemsize = self.nda.itemsize

    def __getattr__(self, name):
        return getattr(self.nda, name)

Very good.  Now note we mustn't override the attribute (`nda`) that holds our memory, so the code forbids it.

In [6]:
grr = GhostArray(5, dtype="int32")
# We don't want to make the GhostArray.nda writable.
grr.nda = np.arange(grr.size, dtype="int32")

AttributeError: attribute 'nda' of '_cython_magic_1b582f42819acefd825559cf89e01f91.GhostArray' objects is not writable

In [ ]:
# Use ellipsis to set the contents.
grr.nda[...] = np.arange(0, grr.size*2, 2, dtype="int32")
print(grr.nda, grr)

## Show C the struct declaration

To build C, we need to leave iPython and play with `distutils`.  Create a directory `01_c_struct/`:

1. `setup.py`: The script driving `distutils`.
2. `ghostbuffer/__init__.py`: Empty file that makes `ghostbuffer` a Python package.
3. `ghostbuffer/core.py`: Python test code.
4. `ghostbuffer/helper.c`: Where our C code goes.
5. `ghostbuffer/gstbuf.pyx`: The Cython module for class `GhostArray`.

If something goes wrong when your edit, check the already-written code in the directory `ref_01_c_struct/`.

Our setup script (`setup.py`) is simple:

```python
from distutils.core import setup
from distutils.extension import Extension

from Cython.Build import cythonize
import numpy as np


def main():
    extensions = cythonize([
        Extension("ghostbuffer.gstbuf",
                  ["ghostbuffer/gstbuf.pyx", "ghostbuffer/helper.c"],
                  include_dirs=[np.get_include()]),
    ])
    setup(
        packages=['ghostbuffer'],
        ext_modules=extensions,
    )


if __name__ == "__main__":
    main()
```

Test code (`ghostbuffer/core.py`) is simple:

```python
from . import gstbuf

def print_int32(num):
    grr = gstbuf.GhostArray(num, creation="arange", dtype="int32")
    grr.print_int32()
```

In `ghostbuffer/helper.c`, add our C helper function:

```c
#include <Python.h>
#include <stdio.h>
#include <numpy/arrayobject.h>
// This header file will be automatically generated by Cython.
#include "gstbuf.h"

void gstbuf_print_int32(gstbuf_t gbuf) {
    if (4 != gbuf.elemsize) {
        return;
    }
    int *elem = (int *)gbuf.elem;
    for (npy_intp jt=0; jt<gbuf.nelem; jt++) {
        printf("%lu: %d\n", jt, elem[jt]);
    }
}
```

In `ghostbuffer/gstbuf.pyx`, fill in the `GhostArray` extension type.  But to let C see the `struct` declaration, we need to make it public:

```python
cdef public:
    ctypedef struct gstbuf_t:
        char *elem
        np.npy_intp nelem
        int elemsize
```

Cython will automatically generate `ghostbuffer/gstbuf.h` header file when it sees the `public` definition.

We also need to let Cython recognize our C helper:

```python
cdef extern:
    void gstbuf_print_int32(gstbuf_t gbuf)
```

Add an interface to the helper function to `GhostArray`:

```python
    def print_int32(self):
        assert "int32" == self.nda.dtype
        gstbuf_print_int32((self._data)[0])
```

## Run it

Now we compile the module:

```bash
$ python setup.py build_ext --inplace
```

and run it:

```bash
$ python -c "from ghostbuffer import core; core.print_int32(10)"
0: 0
1: 1
2: 2
3: 3
4: 4
5: 5
6: 6
7: 7
8: 8
9: 9
```

See the data are correctly shared between Python and C.  Note the only thing copied between the language barrier is the meta-data `gstbuf_t`.

You can play more with it by changing `ghostbuffer.core.print_int32()` to:

```python
def print_int32(num):
    grr = gstbuf.GhostArray(num, creation="arange", dtype="int32")
    from numpy import random
    random.shuffle(grr.nda)
    grr.print_int32()
```

The results:

```bash
$ python -c "from ghostbuffer import core; core.print_int32(10)"
0: 7
1: 8
2: 1
3: 4
4: 0
5: 2
6: 9
7: 6
8: 5
9: 3
```

# Step 2: Go multiple dimension

Create another package in the directory `02_multiple_dimension/`.  Some files can be reused from `01_c_struct`:

1. `setup.py`: The script driving `distutils`.
2. `ghostbuffer/__init__.py`: Empty file that makes `ghostbuffer` a Python package.

Other files need change:

1. `ghostbuffer/gstbuf.pyx`: The Cython module for class `GhostArray`.
2. `ghostbuffer/helper.c`: Where our C code goes.
3. `ghostbuffer/core.py`: Python test code.

If something goes wrong when your edit, check the already-written code in the directory `ref_02_multiple_dimension/`.

## `ghostbuffer/gstbuf.pyx`

To make our array support multiple dimension, we need to remember the shape.  Add two fields in the `gstbuf_t`:

```python
cdef public:
    ctypedef struct gstbuf_t:
        char *elem
        np.npy_intp *shape # A small array holding the shape.
        np.npy_intp nelem
        int ndim # The number of dimensions. Also the length of shape array.
        int elemsize
```

In `GhostArray.__cinit__()`, allocate the new field:

```python
    def __cinit__(self, *args, **kw):
        self._data = <gstbuf_t*>malloc(sizeof(gstbuf_t))
        self._data.elem = <char*>(NULL)
        self._data.shape = <np.npy_intp*>(NULL) # Give it NULL.
        self._data.ndim = 0 # Set it to zero.
        self._data.elemsize = 0
```

In `GhostArray.__dealloc__()`, free the memory:

```python
    def __dealloc__(self):
        if NULL != self._data:
            if NULL != self._data.shape: # Free the new field.
                free(self._data.shape)
            free(self._data)
```

In `GhostArray.__init__()`, add new code to initialize the fields:

```python
    def __init__(self, *args, **kw):
        # ... omit old code for brevity ...
        # Initialize internal data.
        ## elem.
        cdef np.ndarray cnda = self.nda
        self._data.elem = <char*>cnda.data
        ## NEW: shape (just a duplication of PyArrayObject.dimensions).
        if NULL != self._data.shape:
            free(self._data.shape)
        self._data.shape = <np.npy_intp*>malloc(sizeof(np.npy_intp)*ndim)
        for it in range(ndim):
            self._data.shape[it] = self.nda.shape[it]
        ## nelem.
        self._data.nelem = self.nda.size
        ## NEW: ndim (just a duplication of PyArrayObject.nd).
        self._data.ndim = ndim
        ## elemsize (just a duplication of PyArray_Descr.elsize).
        self._data.elemsize = self.nda.itemsize
```

Tell Cython we have a new C helper function:

```python
cdef extern:
    void gstbuf_print_int32(gstbuf_t gbuf)
    void gstbuf_print_int32_md(gstbuf_t gbuf) # New helper.
```

Add related function and properties to `GhostArray`:

```python
    @property
    def nelem(self):
        return self._data.nelem

    @property
    def ndim(self):
        return self._data.ndim

    def print_int32_md(self):
        assert "int32" == self.nda.dtype
        gstbuf_print_int32_md((self._data)[0])
```

## `ghostbuffer/helper.c`

In `ghostbuffer/helper.c`, add our C helper:

```c
void gstbuf_print_int32_md(gstbuf_t gbuf) {
    if (4 != gbuf.elemsize) {
        return;
    }
    npy_intp its[gbuf.ndim];
    for (int it=0; it<gbuf.ndim; ++it) {
        its[it] = 0;
    }
    int *elem = (int *)gbuf.elem;
    for (npy_intp jt=0; jt<gbuf.nelem; jt++) {
        for (int it=0; it<gbuf.ndim; ++it) {
            printf("%lu ", its[it]);
        }
        printf(": %d\n", elem[jt]);
        ++its[gbuf.ndim-1];
        for (int it=gbuf.ndim-1; it>=0; --it) {
            if (its[it] == gbuf.shape[it]) {
                its[it] = 0;
                if (it != 0) { // Carry.
                    ++its[it-1];
                }
            }
        }
    }
}
```

## `ghostbuffer/core.py`

Add the new test code added to `ghostbuffer/core.py`:

```python
def print_int32_md(shape):
    grr = gstbuf.GhostArray(shape, creation="empty", dtype="int32")
    # Note we place an index trick.
    grr.nda[...] = np.arange(grr.nda.size, dtype="int32")[::-1].reshape(shape)
    grr.print_int32_md()
    print(grr.nda)
```

Compile and run it:

```
$ python setup.py build_ext --inplace
$ python -c "from ghostbuffer import core; core.print_int32_md((3,2))"
0 0 : 5
0 1 : 4
1 0 : 3
1 1 : 2
2 0 : 1
2 1 : 0
[[5 4]
 [3 2]
 [1 0]]
```

Play with it more: set to a "normal", incremental array:

```python
def print_int32_md(shape):
    grr = gstbuf.GhostArray(shape, creation="empty", dtype="int32")
    # Note we place an index trick.
    grr.nda[...] = np.arange(grr.nda.size, dtype="int32").reshape(shape)
    grr.print_int32_md()
    print(grr.nda)
```

See the results:

```
$ python -c "from ghostbuffer import core; core.print_int32_md((3,2))"
0 0 : 0
0 1 : 1
1 0 : 2
1 1 : 3
2 0 : 4
2 1 : 5
[[0 1]
 [2 3]
 [4 5]]
```

# Step 3: Ghost index

Create another package in the directory `03_ghost_index/`.  Some files can be reused from `02_multiple_dimension/`:

1. `setup.py`: The script driving `distutils`.
2. `ghostbuffer/__init__.py`: Empty file that makes `ghostbuffer` a Python package.

Other files need change:

1. `ghostbuffer/gstbuf.pyx`: The Cython module for class `GhostArray`.
2. `ghostbuffer/helper.c`: Where our C code goes.
3. `ghostbuffer/core.py`: Python test code.

If something goes wrong when your edit, check the already-written code in the directory `ref_03_ghost_index/`.

## What is "ghost index"

In fortran, it's convenient to declare arrays with negative indices:

```fortran
INTEGER IARR(-3:8, 0:5) ! Rank-two array.
```

In C pointers work just like that.  The following code means getting 8 bytes _before_ the array head `elem`:

```c
double *elem;
double val = elem[-1];
```

The negative index saves a lot of conditional branches when coding array-based numerical methods.  Because this trick works as a "ghost cell" treatment of unstructured meshes, let's call the negative index "ghost index".

Note, negative index in Python (and NumPy) works differently: counting from the end of the container.

## `ghostbuffer/gstbuf.pyx`

We need an additional field in `gstbuf_t` to track the ghost indices:

```python
cdef public:
    ctypedef struct gstbuf_t:
        char *elem
        np.npy_intp *shape
        np.npy_intp *drange # Dimension range: each dimension has a (nghost,nbody) pair.
        np.npy_intp nelem
        int ndim
        int elemsize
```

In `GhostArray.__cinit__()`, allocate the new field:

```python
    def __cinit__(self, *args, **kw):
        self._data = <gstbuf_t*>malloc(sizeof(gstbuf_t))
        self._data.elem = <char*>(NULL)
        self._data.shape = <np.npy_intp*>(NULL)
        self._data.drange = <np.npy_intp*>(NULL) # Give it NULL.
        self._data.ndim = 0
        self._data.elemsize = 0
```

In `GhostArray.__dealloc__()`, free the memory:

```python
    def __dealloc__(self):
        if NULL != self._data:
            if NULL != self._data.shape:
                free(self._data.shape)
            if NULL != self._data.drange: # Free the new field.
                free(self._data.drange)
            free(self._data)
```

In `GhostArray.__init__()`, add new code to initialize the fields:

```python
    def __init__(self, *args, **kw):
        # Pop all custom keyword arguments.
        gshape = kw.pop("gshape", None) # NEW: We need to track the ghost shape.
        creator_name = kw.pop("creation", "empty")
        # ... omit old code for brevity ...
        # Initialize internal data.
        drange = self._calc_drange(self.nda.shape, gshape) # NEW: Calculate data range.
        ## elem.
        cdef np.ndarray cnda = self.nda
        # NEW: Calculate the offset of the C array head elem.
        cdef char *ndhead = <char*>cnda.data
        offset = self._calc_offset(drange) # Call a NEW helper method.
        ndhead += self.nda.itemsize * offset # Offset the element pointer.
        self._data.elem = ndhead
        ## shape (just a duplication of PyArrayObject.dimensions).
        if NULL != self._data.shape:
            free(self._data.shape)
        self._data.shape = <np.npy_intp*>malloc(sizeof(np.npy_intp)*ndim)
        for it in range(ndim):
            self._data.shape[it] = self.nda.shape[it]
        ## NEW: drange.
        if NULL != self._data.drange:
            free(self._data.drange)
        self._data.drange = <np.npy_intp*>malloc(sizeof(np.npy_intp)*ndim*2)
        for it, (nghost, nbody) in enumerate(drange):
            self._data.drange[it*2  ] = nghost
            self._data.drange[it*2+1] = nbody
        ## nelem.
        self._data.nelem = self.nda.size
        ## ndim (just a duplication of PyArrayObject.nd).
        self._data.ndim = ndim
        ## elemsize (just a duplication of PyArray_Descr.elsize).
        self._data.elemsize = self.nda.itemsize
```

`GhostArray.__init__()` used two new helper methods:

```python
    @staticmethod
    def _calc_drange(shape, gshape):
        ndim = len(shape)
        # Make sure gshape is a list.
        if isinstance(gshape, int):
            gshape = [gshape]
        elif None is gshape:
            gshape = [0] * ndim
        gshape = list(gshape)
        # Get gshape correct length.
        if len(gshape) < ndim:
            gshape += [0] * (ndim - len(gshape))
        elif len(gshape) > ndim:
            gshape = gshape[:ndim]
        # Make sure gshape is all positive.
        gshape = [max(0, ng) for ng in gshape]
        # Make sure elements in gshape doesn't exceed those in shape.
        gshape = [min(ng, na) for (ng, na) in zip(gshape, shape)]
        # Build drange.
        drange = [(ng, na-ng) for ng, na in zip(gshape, shape)]
        # Return.
        return drange
    @staticmethod
    def _calc_offset(drange):
        shape = [ng+nb for (ng, nb) in drange]
        ndim = len(drange)
        offset = 0
        for it, (nghost, nbody) in enumerate(drange):
            if ndim-1 == it:
                trail = 1
            else:
                trail = np.multiply.reduce(shape[it+1:])
            offset += nghost * trail
        return offset
```

Associated with the ghost index, we add a bunch of helper properties of `GhostArray` class:

```python
    @property
    def drange(self):
        return tuple((self._data.drange[it*2], self._data.drange[it*2+1])
            for it in range(self._data.ndim))

    @property
    def gshape(self):
        return tuple(self._data.drange[it*2] for it in range(self._data.ndim))

    @property
    def bshape(self):
        return tuple(self._data.drange[it*2+1] for it in range(self._data.ndim))

    @property
    def offset(self):
        return self._calc_offset(self.drange)

    @property
    def _ghostaddr(self):
        cdef np.ndarray cnda = self.nda
        return <unsigned long>(cnda.data)

    @property
    def _bodyaddr(self):
        return <unsigned long>(self._data.elem)
```

Because `gstbuf_t.elem` can now point to anywhere in the middle of the allocated memory, our previous example make break down.  Add an assertion to detect non-applicable scenarios:

```python
    def print_int32(self):
        # Our previous example worked only for non-ghost arrays.
        assert self._ghostaddr == self._bodyaddr
        assert "int32" == self.nda.dtype
        gstbuf_print_int32((self._data)[0])

    def print_int32_md(self):
        # Our previous example worked only for non-ghost arrays.
        assert self._ghostaddr == self._bodyaddr
        assert "int32" == self.nda.dtype
        gstbuf_print_int32_md((self._data)[0])
```

A `GhostArray` is "separable" if only its first dimension has ghost index:

```python
    @property
    def is_separable(self):
        cdef int ndim = self._data.ndim
        if ndim == 1:
            return True
        cdef int it = 1
        while it < ndim:
            if self._data.drange[it*2] != 0:
                return False
            it += 1
        return True

    @property
    def ghostpart(self):
        if not self.is_separable:
            raise ValueError("malformed ghost shape")
        return self.nda[self.gshape[0]-1::-1,...]

    @property
    def bodypart(self):
        if not self.is_separable:
            raise ValueError("malformed ghost shape")
        return self.nda[self.gshape[0]:,...]
```

The last thing we add to `GhostArray` is the API to a demonstrative C helper function:

```python
    def ranged_fill(self):
        assert self.is_separable
        assert "int32" == self.nda.dtype
        gstbuf_ranged_fill((self._data)[0])
```

Make Cython recognize the C function:

```python
cdef extern:
    void gstbuf_print_int32(gstbuf_t gbuf)
    void gstbuf_print_int32_md(gstbuf_t gbuf)
    void gstbuf_ranged_fill(gstbuf_t gbuf) # NEW guy.
```

## `ghostbuffer/helper.c`

To proceed, add the C helper function to `ghostbuffer/helper.c`:

```c
void gstbuf_ranged_fill(gstbuf_t gbuf) {
    size_t nelem;
    int *elem = (int *)gbuf.elem;
    if (4 != gbuf.elemsize)
        return; // Do nothing.
    // Ghost part.
    nelem = gbuf.drange[0];
    for (int it=1; it<gbuf.ndim; it++)
        nelem *= gbuf.drange[it*2] + gbuf.drange[it*2+1];
    for (long it=-1; it>=-nelem; it--)
        elem[it] = it;
    // Body part.
    nelem = gbuf.drange[1];
    for (int it=1; it<gbuf.ndim; it++)
        nelem *= gbuf.drange[it*2] + gbuf.drange[it*2+1];
    for (long it=0; it<nelem; it++)
        elem[it] = it;
}
```

## `ghostbuffer/core.py`

Finally, add the test code to `ghostbuffer/core.py`:

```python
def print_parts(shape, gshape):
    grr = gstbuf.GhostArray(shape, gshape=gshape, dtype="int32")
    grr.ranged_fill()
    print("Do I hold a correct pointer?",
        "Yes" if grr._bodyaddr == grr._ghostaddr + grr.itemsize * grr.offset
        else "No")
    print("The whole array:\n", grr.nda)
    print("The ghost part (first dimension reversed):\n", grr.ghostpart)
    print("The body part:\n", grr.bodypart)
```

Build and run it to see the results:

```
$ python setup.py build_ext --inplace
$ python -c "from ghostbuffer import core; core.print_parts(shape=(3,2), gshape=2)"
Do I hold a correct pointer? Yes
The whole array:
 [[-4 -3]
 [-2 -1]
 [ 0  1]]
The ghost part (first dimension reversed):
 [[-2 -1]
 [-4 -3]]
The body part:
 [[0 1]]
```

# The interesting things

1. Tweaking multi-dimensional array in C is painful.  Adding "ghost" make it worse.
2. In real world you want to shared data structures more complex than that.  Yay.
3. The point is, when you need to go through the pain, you can offset something to Cython rather than do everything in C.
4. Keep [Boost.Python](http://www.boost.org/doc/libs/1_58_0/libs/python/doc/) in mind.
5. NumPy striding is so convenient.  Recall:

    ```python
        @property
        def ghostpart(self):
            if not self.is_separable:
                raise ValueError("malformed ghost shape")
            return self.nda[self.gshape[0]-1::-1,...]
    ```